In [0]:
import os

In [0]:
import os

In [0]:
import pandas as pd

In [0]:
os.system("pip install -r https://raw.githubusercontent.com/George-Michael-Dagogo/World_news_tutorial/main/requirements.txt")

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


0

In [0]:
from newspaper import Article, Config

In [0]:
from nltk.corpus import stopwords

In [0]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import date, timedelta

In [0]:
from datetime import date, timedelta

In [0]:
from newsapi import NewsApiClient

In [0]:
from nltk.tokenize import word_tokenize

In [0]:
# Function to extract content from a URL
def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config=config)

    try:
        page.download()
        page.parse()
        return page.text
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'

In [0]:
%python
def count_words_without_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return len(filtered_words)

def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound'], sentiment_scores['compound']

def full_content(url):
    # Dummy implementation for the full_content function
    return "This is a dummy content for the URL."

def extract_transform_function():
    today = date.today()                        
    yesterday = today - timedelta(days=1)     
    day_before_yesterday = today - timedelta(days=2)   
    
    # Initialize the News API client with an API key
    newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

    top_headlines = newsapi.get_top_headlines(category='entertainment',language='en',page_size=90,page=1)

    articles = top_headlines.get('articles', [])  # Extract articles from the API response

    # Creating DF from the articles, selecting specific columns
    init_df = pd.DataFrame(articles, columns=['source', 'title', 'publishedAt', 'author', 'url'])

    init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

    init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])
    filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | 
                          (init_df['publishedAt'].dt.date == yesterday)]
    filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
    
    df = filtered_df.copy()  # Copy of filtered DF

    df['content'] = df['url'].apply(full_content)  # Apply the full_content function to each URL in the DataFrame
    
    df['content'] = df['content'].str.replace('\n', ' ')  # Replace newlines in the 'content' column with spaces
    
    df = df[df['content'] != 'couldnt retrieve']  # Filter out rows where the content could not be retrieved

    # Download the NLTK stopwords dataset and other required datasets
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')

    # Apply the word count function to the 'content' column
    df['word_count'] = df['content'].apply(count_words_without_stopwords)

    
    nltk.download('vader_lexicon')

    
    sid = SentimentIntensityAnalyzer()    # Initialize the SentimentIntensityAnalyzer

    # Apply the sentiment analysis function to the 'content' column
    df[['sentiment', 'compound_score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))

    

    return df
        
dataframe = extract_transform_function()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...


In [0]:
# Function to count words excluding stopwords
def count_words_without_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return len(filtered_words)

# Function for sentiment analysis
def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound'], sentiment_scores['compound']

def full_content(url):
    # Dummy implementation for the full_content function
    return "This is a dummy content for the URL."

# Data extraction and transformation
def extract_transform_function():
    today = date.today()                        
    yesterday = today - timedelta(days=1)     
    day_before_yesterday = today - timedelta(days=2)   
    
    # Initialize News API client
    newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

    top_headlines = newsapi.get_top_headlines(category='entertainment', language='en', page_size=90, page=1)

    articles = top_headlines.get('articles', [])  # Extract articles

    # Convert articles to DataFrame
    init_df = pd.DataFrame(articles, columns=['source', 'title', 'publishedAt', 'author', 'url'])

    # Extract source name safely
    init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

    # Convert 'publishedAt' to date format and filter
    init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])
    filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | 
                          (init_df['publishedAt'].dt.date == yesterday)]
    
    filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
    
    df = filtered_df.copy()  # Copy DataFrame
    df['content'] = df['url'].apply(full_content)  # Fetch article content
    
    # Clean text formatting
    df['content'] = df['content'].str.replace('\n', ' ')
    
    # Remove entries where content couldn't be retrieved
    df = df[df['content'] != 'couldnt retrieve']

    # Download required NLTK datasets
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('vader_lexicon')

    # Calculate word countl
    df['word_count'] = df['content'].apply(count_words_without_stopwords)


    # Perform sentiment analysis
    df[['sentiment', 'compound_score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))


    return df

# Extract transformed data
dataframe = extract_transform_function()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-d6dd3ab5-01d5-4cb5-8f96-cc/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
dataframe.display()

source,title,date_posted,author,url,content,word_count,sentiment,compound_score
Cageside Seats,"Rumor Roundup: Becky Lynch return, Bayley sympathy, Rey Mysterio injury, AAA, more! - Cageside Seats",2025-04-21T05:00:00.000Z,Randall Ortman,https://www.cagesideseats.com/2025/4/21/24412839/rumor-roundup-april-21-2025-becky-lynch-return-bayley-sympathy-rey-mysterio-injury-aaa-deal,This is a dummy content for the URL.,4,0.0,0.0
CBS Sports,"WWE WrestleMania 41 results, grades: John Cena wins record 17th world title, ends Cody Rhodes' year-long reign - CBS Sports",2025-04-21T03:58:50.000Z,Brent Brookhouse,https://www.cbssports.com/wwe/news/wwe-wrestlemania-41-results-grades-john-cena-wins-record-17th-world-title-ends-cody-rhodes-year-long-reign/live/,This is a dummy content for the URL.,4,0.0,0.0
Variety,Pedro Pascal Speaks Out After ‘The Last of Us’ Twist and Says ‘I’m in Active Denial’; Bella Ramsey ‘Sobbed’ Reading It: A ‘Brilliant Achievement’ If It’s ‘Painful for People’ - Variety,2025-04-21T03:36:00.000Z,Zack Sharf,https://variety.com/2025/tv/news/pedro-pascal-bella-ramsey-joel-death-last-of-us-1236373640/,This is a dummy content for the URL.,4,0.0,0.0
Just Jared,"Kristen Stewart & Dylan Meyer Tie the Knot on Easter Sunday, Wedding Details Revealed! - Just Jared",2025-04-21T03:26:51.000Z,null,https://www.justjared.com/2025/04/20/kristen-stewart-dylan-meyer-tie-the-knot-on-easter-sunday-wedding-details-revealed/,This is a dummy content for the URL.,4,0.0,0.0
Yahoo Entertainment,"WWE WrestleMania 41 Night 2 live results: Updates, winners and losers, highlights, grades and analysis - Yahoo Sports",2025-04-21T02:37:30.000Z,Anthony Sulla-Heffinger,https://sports.yahoo.com/wrestling/live/wwe-wrestlemania-41-night-2-results-grades-analysis-john-cena-beats-cody-rhodes-in-lackluster-headliner-the-rock-no-shows-080034111.html,This is a dummy content for the URL.,4,0.0,0.0
CBS Sports,"'Stone Cold' Steve Austin crashes ATV into barricade at WWE WrestleMania 41, nearly hits fan at ringside - CBS Sports",2025-04-21T02:27:26.000Z,,https://www.cbssports.com/wwe/news/stone-cold-steve-austin-crashes-atv-into-barricade-at-wwe-wrestlemania-41-nearly-hits-fan-at-ringside/,This is a dummy content for the URL.,4,0.0,0.0
Nodq.com,Results of AJ Styles vs. Logan Paul at WWE WrestleMania 41 Sunday - NoDQ.com,2025-04-21T01:18:09.000Z,Aaron Rift,https://nodq.com/news/results-of-aj-styles-vs-logan-paul-at-wwe-wrestlemania-41-sunday/,This is a dummy content for the URL.,4,0.0,0.0
Cageside Seats,TNA Champion answers Randy Orton’s open challenge at WrestleMania 41 - Cageside Seats,2025-04-21T00:57:44.000Z,Geno Mrosko,https://www.cagesideseats.com/wwe/2025/4/20/24412664/wrestlemania-41-results-tna-champion-joe-hendry-randy-orton-open-challenge,This is a dummy content for the URL.,4,0.0,0.0
Hollywood Reporter,Bowen Yang Says Aimee Lou Wood’s Criticism of ‘White Lotus’ Parody on ‘SNL’ Is “Completely Valid” - The Hollywood Reporter,2025-04-21T00:56:40.000Z,Carly Thomas,http://www.hollywoodreporter.com/tv/tv-news/bowen-yang-reacts-aimee-lou-wood-criticism-white-lotus-snl-1236196342/,This is a dummy content for the URL.,4,0.0,0.0
Vulture,Elizabeth Hurley and Billy Ray Cyrus Hard Launch With Easter IG Post - Vulture,2025-04-21T00:53:50.000Z,Bethy Squires,http://www.vulture.com/article/elizabeth-hurley-billy-ray-cyrus-dating.html,This is a dummy content for the URL.,4,0.0,0.0


In [0]:
print(dataframe.dtypes)

source                         object
title                          object
date_posted       datetime64[ns, UTC]
author                         object
url                            object
content                        object
word_count                      int64
sentiment                     float64
compound_score                float64
dtype: object


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS the_news;
CREATE TABLE IF NOT EXISTS the_news.news_table (
source STRING,
title STRING,
date_posted DATE,
author STRING,
url STRING,
content STRING,
word_count INT,
sentiment STRING,
compound_score DOUBLE
)

In [0]:
%sql DESCRIBE FORMATTED the_news.news_table;


col_name,data_type,comment
source,string,null
title,string,null
date_posted,date,null
author,string,null
url,string,null
content,string,null
word_count,int,null
sentiment,string,null
compound_score,double,null
,,


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, DoubleType

# Initialize Spark session
spark = SparkSession.builder.appName("CreateTableExample").getOrCreate()

# Convert sentiment to string for Spark compatibility
dataframe['sentiment'] = dataframe['sentiment'].fillna('').astype(str)

# Define schema explicitly
schema = StructType([
    StructField("source", StringType(), True),
    StructField("title", StringType(), True),
    StructField("date_posted", DateType(), True), 
    StructField("author", StringType(), True),
    StructField("url", StringType(), True),
    StructField("content", StringType(), True),
    StructField("word_count", IntegerType(), True),
    StructField("sentiment", StringType(), True),
    StructField("compound_score", DoubleType(), True)
])

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(dataframe, schema=schema)

spark_df.write.mode('append').saveAsTable('the_news.news_table')


In [0]:
%sql
select * from the_news.news_table

source,title,date_posted,author,url,content,word_count,sentiment,compound_score
Cageside Seats,"WWE NXT Stand & Deliver 2025 results, live streaming match coverage - Cageside Seats",2025-04-19,Claire Elizabeth,https://www.cagesideseats.com/wwe/2025/4/19/24410997/wwe-nxt-stand-deliver-2025-results-live-streaming-match-coverage,This is a dummy content for the URL.,4,0.0,0.0
CNN,Why the 20-year-old ‘Pride & Prejudice’ film still bewitches audiences today - CNN,2025-04-19,Leah Asmelash,https://www.cnn.com/2025/04/19/entertainment/pride-prejudice-film-20th-anniversary-cec/index.html,This is a dummy content for the URL.,4,0.0,0.0
Variety,"Box Office: ‘Sinners’ Stokes $19.2 Million Opening Day, ‘A Minecraft Movie’ Still Looks to Block Off No. 1 in Third Weekend - Variety",2025-04-19,J. Kim Murphy,https://variety.com/2025/film/news/box-office-sinners-opening-day-minecraft-no1-1236372989/,This is a dummy content for the URL.,4,0.0,0.0
Hollywood Reporter,Harvey Weinstein’s Last Stand - The Hollywood Reporter,2025-04-19,Winston Cho,http://www.hollywoodreporter.com/news/general-news/harvey-weinstein-new-trial-1236190862/,This is a dummy content for the URL.,4,0.0,0.0
Bleacher Report,"Final Picks for Cena, Rhodes, Rock, Reigns and WWE WrestleMania 41 Match Card - Bleacher Report",2025-04-19,Donald Wood,https://bleacherreport.com/articles/25187437-final-picks-cena-rhodes-rock-reigns-and-wwe-wrestlemania-41-match-card,This is a dummy content for the URL.,4,0.0,0.0
Politico,The WrestleMania Presidency - Yahoo Home,2025-04-19,Zach Montellaro,https://www.politico.com/news/magazine/2025/04/19/donald-trump-pro-wrestling-wwe-00298303,This is a dummy content for the URL.,4,0.0,0.0
Wrestletalk.com,What Happened In Triple H WWE Hall Of Fame 2025 Hour Long Speech - WrestleTalk,2025-04-19,Sanchez Taylor,https://wrestletalk.com/news/triple-h-wwe-hall-of-fame-2025-speech/,This is a dummy content for the URL.,4,0.0,0.0
Variety,Lady Gaga Apologizes for Mic Malfunctions During Coachella Weekend 2 Set: ‘At Least You Know I Sing Live’ - Variety,2025-04-19,Thania Garcia,https://variety.com/2025/music/news/lady-gaga-apologizes-broken-mic-coachella-weekend-2-i-sing-live-1236373171/,This is a dummy content for the URL.,4,0.0,0.0
Tom's Guide,"5 best new movies to stream this weekend on Netflix, Hulu, Max, and more - Tom's Guide",2025-04-19,Alyse Stanley,https://www.tomsguide.com/entertainment/streaming/movies-to-stream-this-weekend-april-18-2025,This is a dummy content for the URL.,4,0.0,0.0
Polygon,"Star Wars Zero Company, a turn-based game at the end of the Clone Wars, gets full reveal - Polygon",2025-04-19,Maddy Myers,https://www.polygon.com/star-wars/560074/star-wars-zero-company-xcom-game-reveal,This is a dummy content for the URL.,4,0.0,0.0
